In [52]:
import numpy as np
import math
import csv 
import matplotlib.pyplot as plt
import matplotlib.tri as mtri
import seaborn as sns
import os
import imageio
import colorcet as cc
import cartopy.crs as ccrs
import stripy

%matplotlib inline
%config InlineBackend.figure_format = 'svg'

sns.set(rc={"lines.linewidth": 2.5,"figure.figsize":(12,6),"lines.markersize":2},font_scale=1.25)
sns.set_context("notebook")
sns.set_style("white")

In [2]:
def cart2sph(x, y, z):
    r = np.sqrt(np.power(x, 2) + np.power(y, 2) + np.power(z, 2))
    colat = np.arctan2(np.sqrt(np.power(x, 2) + np.power(y, 2)), z)
    lon = np.arctan2(y, x)
    return r, colat, lon

def latlon(x, y, z): # returns latitude and longitude in degrees 
    r, colat, lon = cart2sph(x, y, z)
    return lon, colat

In [175]:
points_count = np.zeros(101)

In [177]:
with open('fast_point_counts_rh4_10242_noamr.csv', 'r', newline='') as csvfile1:
    reader1 = csv.reader(csvfile1, delimiter=',', quotechar='|', quoting=csv.QUOTE_NONNUMERIC)
    i = 0
    for row in reader1:
#         print(row)
        points_count[i] = row[0]
        i += 1

In [178]:
data = np.zeros((101, int(points_count.max()), 8)) # x y z vorticity passive tracer area lon lat

In [179]:
with open('fast_output_rh4_10242_noamr.csv', 'r', newline='') as csvfile:
    reader = csv.reader(csvfile, delimiter=',', quotechar='|', quoting=csv.QUOTE_NONNUMERIC)
    i = 0
    t = 0
    for row in reader:
#         t = i // point_count
#         point = i % point_count
        data[t, i, 0:6] = row
        lon, lat = latlon(*row[0:3])
        data[t, i, 6:8] = lon, lat
        i += 1
        if (i >= points_count[t]):
            t += 1
            i = 0
        if t >= 101:
            break

In [180]:
lons = data[-1,:,7]
lats = data[-1,:,6]
areas = data[-1,:,5]
vor = data[-1,:,3]
# true_vor = 2 * math.pi / 7 * np.sin(lats) + 30 * np.sin(lats) * np.power(np.cos(lats), 4) * np.cos(4 * lons)
true_vor=data[0,:,3]

In [181]:
np.sqrt(np.sum(np.square(true_vor - vor) * areas) / np.sum(np.square(true_vor) * areas))

2.135743711960883

In [163]:
np.sum(np.square(true_vor) * areas)

0.0

In [183]:
areas

array([0.00094882, 0.00094882, 0.00094882, ..., 0.00114763, 0.00114763,
       0.00113898])

In [182]:
data

array([[[ 0.00000000e+00,  0.00000000e+00,  1.00000000e+00, ...,
          9.48819000e-04,  0.00000000e+00,  0.00000000e+00],
        [ 8.94427000e-01,  0.00000000e+00,  4.47214000e-01, ...,
          9.48819000e-04,  0.00000000e+00,  1.10714827e+00],
        [ 7.23607000e-01,  5.25731000e-01, -4.47214000e-01, ...,
          9.48819000e-04,  6.28318296e-01,  2.03444425e+00],
        ...,
        [ 6.47096000e-01,  6.17167000e-01, -4.47628000e-01, ...,
          1.14763000e-03,  7.61729503e-01,  2.03490752e+00],
        [ 6.89761000e-01, -6.17167000e-01, -3.78595000e-01, ...,
          1.14763000e-03, -7.29909659e-01,  1.95907425e+00],
        [ 7.45250000e-01, -5.16324000e-01, -4.21915000e-01, ...,
          1.13898000e-03, -6.05890805e-01,  2.00635275e+00]],

       [[ 0.00000000e+00,  0.00000000e+00,  1.00000000e+00, ...,
          9.48819000e-04,  0.00000000e+00,  0.00000000e+00],
        [ 8.94427000e-01,  0.00000000e+00,  4.47214000e-01, ...,
          9.48819000e-04,  0.00000000e